# Installing Dependencies

In [ ]:
#Installing all dependencies needed
!pip install labelme opencv-python matplotlib

In [ ]:
#Displays everything installed 
!pip list

# Label Images

In [ ]:
#Used for labeling the images in the dataset
!labelme

# Construct Image Loading Function

## Import Dependencies & Check GPU

In [ ]:
# Import TensorFlow for building and running the deep learning model
import tensorflow as tf
# Import OpenCV for image and video processing
import cv2
# Import time module to measure time intervals
import time
# Import json to handle JSON files
import json
# Import os to interact with the operating system 
import os
# Import NumPy for numerical operations
import numpy as np
# Import pyplot from matplotlib for plotting and displaying images
from matplotlib import pyplot as plt
# Import PIL Image for additional image manipulation options
from PIL import Image
# Import datetime to get the current date and time 
from datetime import datetime

In [ ]:
# List all physical GPU devices recognized by TensorFlow
gpus = tf.config.experimental.list_physical_devices('GPU')
# Loop through each GPU detected to configure its memory growth setting
for gpu in gpus:
    # Enable memory growth for the GPU to allocate memory as needed, preventing TensorFlow from using all GPU memory at once
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
#Outputs availability of GPU
tf.test.is_gpu_available()

## Load Image into Data Pipeline

In [ ]:
#Loads an image and resizes the tensor to 224x224x3

# Image dimensions
IMG_HEIGHT = 224  
IMG_WIDTH = 224   

def load_image(x):
    # Read the image file as a byte string
    byte_img = tf.io.read_file(x)
    # Decode the byte string into an image tensor with 3 channels (RGB)
    img = tf.image.decode_image(byte_img, channels=3)
    # Setting shape of the image for 3 channels
    img.set_shape([None, None, 3])  
    # Resize the image to (224x224)
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])  
    
    # Returning preprocessed image
    return img

#  Image Augmentation for Images & Labels 

In [ ]:
#Applying the following augmentations to the dataset

"""
 - HorizontalFlip: Flips the image horizontally.
 - RandomBrightnessContrast: Randomly adjusts the brightness and contrast.
 - RandomGamma: Randomly applies gamma correction.
 - RGBShift: Randomly shifts the RGB channels.
 - VerticalFlip: Flips the image vertically.
 """

#Importing image augmentation library
import albumentations as alb
augmentor = alb.Compose([alb.HorizontalFlip(p=0.5),
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2),
                         alb.RGBShift(p=0.2),
                         alb.VerticalFlip(p=0.5)],
                         bbox_params = alb.BboxParams(format = 'albumentations', label_fields = ['class_labels']))

#albumentations format = normalized[x_min, y_min, x_max, y_max]

In [ ]:
#Getting example image for testing labels
img = cv2.imread(os.path.join('C:\\Users\\wfenn\\ECEN_403\\Data\\Train\\Images', 'baby_soybean_jpeg (4).jpeg'))

In [ ]:
#Extracting dimensions (height and width)
height, width, _ = img.shape

In [ ]:
#Extracting label
with open(os.path.join('C:\\Users\\wfenn\\ECEN_403\\Data\\Train\\Labels', 'baby_soybean_jpeg (4).json'), 'r') as f:
    # Load the JSON content as a dictionary, so it can be accessed as 'label'
    label = json.load(f)

In [ ]:
print("Bounding Box Coordinates:\n", label['shapes'][0]['points'])

In [ ]:
# Initialize a list of four zeros to store bounding box coordinates [x1, y1, x2, y2]
coords = [0, 0, 0, 0]

# Assign the x-coordinate of the first point to the first index of 'coords'
coords[0] = label['shapes'][0]['points'][0][0]
# Assign the y-coordinate of the first point to the second index of 'coords'
coords[1] = label['shapes'][0]['points'][0][1]
# Assign the x-coordinate of the second point to the third index of 'coords'
coords[2] = label['shapes'][0]['points'][1][0]
# Assign the y-coordinate of the second point to the fourth index of 'coords'
coords[3] = label['shapes'][0]['points'][1][1]

In [ ]:
coords
#raw pascal_voc format = [x_min, y_min, x_max, y_max]

In [ ]:
#Normalizing the bounding box coordinates 
coords = list(np.divide(coords, [width, height, width, height]))

In [ ]:
coords
#albumentations format = normalized[x_min, y_min, x_max, y_max]

In [ ]:
#Augmenting specific crop image
augmented = augmentor(image = img, bboxes = [coords], class_labels = ['Crop'])

In [ ]:
# Drawing a rectangle on the augmented image using bounding box coordinates
cv2.rectangle(
    augmented['image'],  # The image to draw the rectangle on (augmented image)
    tuple(np.multiply(augmented['bboxes'][0][:2], [width, height]).astype(int)),  # Top-left corner coordinates (scaled to original image size)
    tuple(np.multiply(augmented['bboxes'][0][2:], [width, height]).astype(int)),  # Bottom-right corner coordinates (scaled to original image size)
    (0, 255, 0),  # Color of the rectangle (green in BGR format)
    2)  # Thickness of the rectangle's border (2 pixels)

# Displaying the augmented image with the bounding box using matplotlib
plt.imshow(augmented['image'])  # This will show the image with the drawn rectangle in the Jupyter notebook

# Build Augmentation Pipeline

## Create Augmented Data

In [ ]:
# Define the augmentation pipeline using Albumentations

# Iterate through partitions
for partition in ['Train', 'Test', 'Val']: 
    # Define paths for images, labels, and augmented data
    image_dir = os.path.join('C:\\Users\\wfenn\\ECEN_403\\Datav2', partition, 'Images')
    label_dir = os.path.join('C:\\Users\\wfenn\\ECEN_403\\Datav2', partition, 'Labels')
    aug_image_dir = os.path.join('C:\\Users\\wfenn\\ECEN_403\\Aug_Datav6', partition, 'Images')
    aug_label_dir = os.path.join('C:\\Users\\wfenn\\ECEN_403\\Aug_Datav6', partition, 'Labels')

    # Ensure output directories exist
    os.makedirs(aug_image_dir, exist_ok=True)
    os.makedirs(aug_label_dir, exist_ok=True)

    # Process each image in the current partition
    for image in os.listdir(image_dir):
        img = cv2.imread(os.path.join(image_dir, image))

        # Get the image dimensions
        height, width, _ = img.shape

        # Initialize empty list for bounding boxes and class labels
        bboxes = []
        class_labels = []
        
        # Define path to the corresponding label file
        label_path = os.path.join(label_dir, f'{image.split(".")[0]}.json')

        # Read labels if the label file exists
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label_data = json.load(f)

            # Iterate through all shapes in the label file (multiple bboxes)
            for shape in label_data['shapes']:
                # Normalize bounding box coordinates based on image dimensions
                coords = [
                    shape['points'][0][0] / width,  # x_min
                    shape['points'][0][1] / height,  # y_min
                    shape['points'][1][0] / width,  # x_max
                    shape['points'][1][1] / height   # y_max
                ]
                bboxes.append(coords)

                # Handle missing 'class' key
                if 'class' in shape:
                    class_labels.append(shape['class'])
                else:
                    # Assign labels based on filename if 'class' is missing
                    if 'soybean' in image.lower():
                        class_labels.append(0)  # Assign 0 for crop
                    elif 'weed' in image.lower() or 'rrpw' in image.lower():
                        class_labels.append(1)  # Assign 1 for weed

        try:
            # Set the number of augmentations based on class label
            if 0 in class_labels:  #crop
                augmentations_count = 22
            elif 1 in class_labels:  #weed
                augmentations_count = 10
            else:
                augmentations_count = 0  # No augmentation if class not recognized

            # Perform augmentation
            for x in range(augmentations_count):
                augmented = augmentor(image=img, bboxes=bboxes, class_labels=class_labels)

                # Save augmented image
                augmented_image_path = os.path.join(aug_image_dir, f'{image.split(".")[0]}.{x}.jpg')
                cv2.imwrite(augmented_image_path, augmented['image'])

                # Prepare and save the augmented bounding box annotations
                augmented_annotations = []
                for aug_bbox, aug_class in zip(augmented['bboxes'], augmented['class_labels']):
                    annotation = {
                        "image": f'{image.split(".")[0]}.{x}.jpg', # Augmented image filename
                        "class": aug_class, # Class label
                        "bbox": aug_bbox  # Coordinates in albumentations format
                    }
                    augmented_annotations.append(annotation)

                # Save annotations to JSON file
                augmented_label_path = os.path.join(aug_label_dir, f'{image.split(".")[0]}.{x}.json')
                with open(augmented_label_path, 'w') as f:
                    json.dump(augmented_annotations, f, indent=4)

        except Exception as e: # Handle errors during processing and display the filename and error
            print(f"Error processing {image}: {e}")


In [ ]:
def augment_background_images():
    # Iterate through partitions
    for partition in ['Train', 'Test', 'Val']: 
        # Define paths for images, labels, and augmented data
        image_dir = os.path.join('C:\\Users\\wfenn\\ECEN_403\\Background', partition, 'Images')
        aug_image_dir = os.path.join('C:\\Users\\wfenn\\ECEN_403\\Aug_Datav6', partition, 'Images')
        aug_label_dir = os.path.join('C:\\Users\\wfenn\\ECEN_403\\Aug_Datav6', partition, 'Labels')

        # Set the augmentation count based on the partition
        augmentation_count = 54 if partition == 'Train' else 50

        # Ensure output directories exist
        os.makedirs(aug_image_dir, exist_ok=True)
        os.makedirs(aug_label_dir, exist_ok=True)

        for image in os.listdir(image_dir):
            img = cv2.imread(os.path.join(image_dir, image))

            # Perform augmentation the specified number of times
            for x in range(augmentation_count):
                augmented = augmentor(image=img, bboxes=[], class_labels=[])

                # Save augmented image
                augmented_image_path = os.path.join(aug_image_dir, f'{image.split(".")[0]}.{x}.jpg')
                cv2.imwrite(augmented_image_path, augmented['image'])

                # Save augmented labels
                augmented_annotations = [{
                    "image": f'{image.split(".")[0]}.{x}.jpg',
                    "class": 2,  # Class 2 for background
                    "bbox": [0, 0, 0, 0]  # Use [0, 0, 0, 0] for background
                }]

                # Save to JSON file in the Labels folder
                augmented_label_path = os.path.join(aug_label_dir, f'{image.split(".")[0]}.{x}.json')
                with open(augmented_label_path, 'w') as f:
                    json.dump(augmented_annotations, f, indent=4)

# Run the augmentation function
augment_background_images()


## Load Augmented Images to Dataset

In [ ]:
# converts each image file path into a resized image tensor of 
# shape (224,224,3) with normalized pixel values between (0,1)
train_images = tf.data.Dataset.list_files('C:\\Users\\wfenn\\ECEN_403\\Aug_Datav6\\Train\\Images\\*.jpg', shuffle=False)
# Load each image file and decode it into a tensor with RGB channels
train_images = train_images.map(load_image)
# Explicitly resize each image tensor to (224, 224)
train_images = train_images.map(lambda x: tf.image.resize(x, (224,224)))
# Normalize pixel values to the range [0, 1] by dividing by 255
train_images = train_images.map(lambda x: x/255)

In [ ]:
# converts each image file path into a resized image tensor of 
# shape (224,224,3) with normalized pixel values between (0,1)
test_images = tf.data.Dataset.list_files('C:\\Users\\wfenn\\ECEN_403\\Aug_Datav6\\Test\\Images\\*.jpg', shuffle=False)
# Load each image file and decode it into a tensor with RGB channels
test_images = test_images.map(load_image)
# Explicitly resize each image tensor to (224, 224)
test_images = test_images.map(lambda x: tf.image.resize(x, (224,224)))
# Normalize pixel values to the range [0, 1] by dividing by 255
test_images = test_images.map(lambda x: x/255)

In [ ]:
# converts each image file path into a resized image tensor of 
# shape (224,224,3) with normalized pixel values between (0,1)
val_images = tf.data.Dataset.list_files('C:\\Users\\wfenn\\ECEN_403\\Aug_Datav6\\Val\\Images\\*.jpg', shuffle=False)
# Load each image file and decode it into a tensor with RGB channels
val_images = val_images.map(load_image)
# Explicitly resize each image tensor to (224, 224)
val_images = val_images.map(lambda x: tf.image.resize(x, (224,224)))
# Normalize pixel values to the range [0, 1] by dividing by 255
val_images = val_images.map(lambda x: x/255)

# Prepare Labels

## Build Label Loading Function

In [ ]:
max_classes = 7 

"""
Max number of annotations from a label in the dataset. 
Padding the rest of labels to meet the maximum number of annotations so that 
the number is consistant rather than dealing with variable annotations across each label.
"""

def load_labels(label_path):
    try:
        # Read the JSON file
        label_data = tf.io.read_file(label_path)
        labels = json.loads(label_data.numpy().decode('utf-8'))  # Decode bytes to string

        # Extract classes and bounding boxes
        classes = [item['class'] for item in labels]
        bboxes = [item['bbox'] for item in labels]
        
        # Pad classes list and convert to tensor
        class_tensor = tf.convert_to_tensor(classes + [255] * (max_classes - len(classes)), dtype=tf.uint8)

        # Pad bounding boxes and convert to tensor
        if len(bboxes) == 0: # Background
            # If there are no bounding boxes, fill with padding
            bbox_tensor = tf.convert_to_tensor([[0, 0, 0, 0]] * max_classes, dtype=tf.float32)
        else:
            # Pad bounding boxes
            bbox_tensor = tf.convert_to_tensor(bboxes + [[0, 0, 0, 0]] * (max_classes - len(bboxes)), dtype=tf.float32)
        
        return class_tensor, bbox_tensor

    # Error handling
    except FileNotFoundError:
        print(f"Error: The file {label_path} was not found.")
        return None, None
    except json.JSONDecodeError:
        print(f"Error: The file {label_path} could not be decoded.")
        return None, None


In [ ]:
#Function to test and display annotations
def test_load_labels():
    # Load labels
    classes, bboxes = load_labels('C:\\Users\\wfenn\\ECEN_403\\Aug_Datav6\\Test\\Labels\\baby_soybean_jpeg (3).0.json')

    # Print the results
    print("Class Labels:", classes.numpy())  # Convert tensor to numpy for easy reading
    print("Bounding Boxes: \n", bboxes.numpy())

# Run the test
test_load_labels()

## Load Labels to Dataset

In [ ]:
# Load the label files from the directory as a tf.data.Dataset
train_labels = tf.data.Dataset.list_files('C:\\Users\\wfenn\\ECEN_403\\Aug_Datav6\\Train\\Labels\\*.json', shuffle=False)

# Map over each label file and process them using the `load_labels` function
# Use tf.py_function and set shapes for consistent output
train_labels = train_labels.map(
    lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float32]), # Apply `load_labels` function
    num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Ensure the output shape of each processed label matches the expected dimensions
train_labels = train_labels.map(lambda class_tensor, bbox_tensor: (
    tf.ensure_shape(class_tensor, [max_classes]), #Shape [7]
    tf.ensure_shape(bbox_tensor, [max_classes, 4]))) #Shape [7,4]

In [ ]:
# Load the label files from the directory as a tf.data.Dataset
test_labels = tf.data.Dataset.list_files('C:\\Users\\wfenn\\ECEN_403\\Aug_Datav6\\Test\\Labels\\*.json', shuffle=False)

# Map over each label file and process them using the `load_labels` function
# Use tf.py_function and set shapes for consistent output
test_labels = test_labels.map(
    lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float32]), # Apply `load_labels` function
    num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Ensure the output shape of each processed label matches the expected dimensions
test_labels = test_labels.map(lambda class_tensor, bbox_tensor: (
    tf.ensure_shape(class_tensor, [max_classes]), #Shape [7]
    tf.ensure_shape(bbox_tensor, [max_classes, 4]))) #Shape [7,4]

In [ ]:
# Load the label files from the directory as a tf.data.Dataset
val_labels = tf.data.Dataset.list_files('C:\\Users\\wfenn\\ECEN_403\\Aug_Datav6\\Val\\Labels\\*.json', shuffle=False)

# Map over each label file and process them using the `load_labels` function
# Use tf.py_function to call the custom Python function for each file
val_labels = val_labels.map(
    lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float32]), # Apply `load_labels` function
    num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Ensure the output shape of each processed label matches the expected dimensions
val_labels = val_labels.map(lambda class_tensor, bbox_tensor: (
    tf.ensure_shape(class_tensor, [max_classes]), #Shape [7]
    tf.ensure_shape(bbox_tensor, [max_classes, 4]))) #Shape [7,4]

In [ ]:
# Function to print a small batch from the dataset
def print_dataset_example(dataset, num_examples):
    # Iterate over the dataset and print the specified number of examples
    for classes, bboxes in dataset.take(num_examples):
        print("Classes:", classes.numpy())
        print("Bounding Boxes:", bboxes.numpy())

# Print one example from the `test_labels` dataset
print_dataset_example(test_labels, num_examples=1)


In [ ]:
# Convert the train_labels dataset to a NumPy iterator and fetch the next example
train_labels.as_numpy_iterator().next()

# Combine Labels & Images

In [ ]:
#Printing length of train, test, and val images and labels
len(train_images), len(train_labels), len(test_images), len(test_labels), len(val_images), len(val_labels)

In [ ]:
# Combine the train_images and train_labels datasets into a single dataset
train = tf.data.Dataset.zip((train_images, train_labels))

# Shuffle the dataset with a buffer size of 6000
# Batch the dataset into batches of size 32
# Prefetch the data in the background while the model is training 
train = train.shuffle(buffer_size=6000)
train = train.batch(32)
train = train.prefetch(4)

In [ ]:
# Combine the test_images and test_labels datasets into a single dataset
test = tf.data.Dataset.zip((test_images, test_labels))

# Batch the dataset into batches of size 32 
# Prefetch the data in the background while the model is being evaluated 
test = test.batch(32)
test = test.prefetch(4)

In [ ]:
# Combine the val_images and val_labels datasets into a single dataset
val = tf.data.Dataset.zip((val_images, val_labels))

# Batch the dataset into batches of size 32
# Prefetch the data in the background while the model is being validated 
val = val.batch(32)
val = val.prefetch(4)

## View Images &  Labels

In [ ]:
# Create an iterator for the training dataset
data_samples = train.as_numpy_iterator()

In [ ]:
#Fetching the next batch
res = data_samples.next()

In [ ]:
images = res[0]  # Image batch
classes = res[1][0]  # Padded classes
bboxes = res[1][1]  # Padded bounding boxes

# Set up the figure
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(20, 20))

for ax, img, bbox, cls in zip(axes, images, bboxes, classes):
    # Convert image to BGR format (OpenCV uses BGR)
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    # Draw bounding boxes
    for box, label in zip(bbox, cls):
        if label == 255:  # Skip padded boxes
            continue

        # Rescale, color, & draw bboxes (green for class 0 (crop), red for class 1 (weed))
        x1, y1, x2, y2 = (box * [img.shape[1], img.shape[0], img.shape[1], img.shape[0]]).astype(int)  
        color = (0, 255, 0) if label == 0 else (255, 0, 0) 
        cv2.rectangle(img_bgr, (x1, y1), (x2, y2), color, 2)
    
    # Display the image with bounding boxes on the subplot
    ax.imshow(img_bgr)
    ax.axis('on')
    
# Display the plot with all images and bounding boxes
plt.show()


# Build Deep Learning Model

In [ ]:
from tensorflow.keras.models import Model, Sequential  # Import Model and Sequential classes for creating models
from tensorflow.keras import layers, models  # Import layers and models for building and defining neural network structures
from tensorflow.keras.layers import Input, Conv2D, Dense, Dropout, GlobalMaxPooling2D, Flatten, AveragePooling2D, BatchNormalization, Reshape, Lambda, Activation  
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau  # Import callbacks for model training (e.g., stop early, save model, reduce learning rate)
from tensorflow.keras.regularizers import l1_l2, l2  # Import regularizers to prevent overfitting by adding penalties on model parameters
from tensorflow.keras.applications import VGG16  # Import the pre-trained VGG16 model (used for transfer learning)
from tensorflow.keras.metrics import CategoricalAccuracy  # Import accuracy metric for multi-class classification tasks
from tensorflow.keras.callbacks import Callback  # Import Callback class for creating custom callbacks during training

In [ ]:
#Importing VGG16 without the fully connected layer as model backbone
vgg = VGG16(include_top=False)

In [ ]:
#Shows vgg architecture
vgg.summary()

## Build Instance of Model

In [ ]:
# WIZARDv15 Model 
def build_model(num_boxes=7, num_classes=3): 
    input_layer = Input(shape=(224, 224, 3))  # Define the input layer with shape 224x224x3 (RGB image)

    # Feature Extraction using VGG16 with pre-trained weights
    vgg_base = VGG16(weights='imagenet', include_top=False, input_tensor=input_layer)  # Use VGG16 as a base model for feature extraction, without the fully connected layers
    vgg_base.trainable = False  # Freeze the VGG16 layers to prevent updates during training
    
    vgg_output = vgg_base.output  # Capture the output of VGG16 feature extractor

    # Optionally unfreeze last few layers for fine-tuning
    for layer in vgg_base.layers[-4:]:  # Select the last 4 layers of VGG16 to be trainable
        layer.trainable = True  # Enable training on the selected layers for fine-tuning
    
    # Global Feature Pooling
    f1 = GlobalMaxPooling2D()(vgg_output)  # Apply global max pooling to reduce the spatial dimensions to a single vector
    
    # Classification Head
    class_head = Dense(1024, activation='relu', kernel_regularizer=l1_l2(l1=0.00095, l2=0.0095))(f1)  # Dense layer with L1/L2 regularization for class prediction
    class_head = BatchNormalization()(class_head)  # Normalize the activations to improve training stability
    class_head = Dropout(0.5)(class_head)  # Dropout layer to prevent overfitting in the classification head
    class_head = Dense(num_boxes * num_classes, activation='linear')(class_head)  # Dense layer to predict class scores for each bounding box
    class_head = Reshape((num_boxes, num_classes))(class_head)  # Reshape to (num_boxes, num_classes) for each bounding box
    class_head = Lambda(lambda x: tf.nn.softmax(x, axis=-1), name='classification_output')(class_head)  # Apply softmax activation to get probabilities for each class
    
    # Localization Head
    regress_head = Dense(1024, activation='relu', kernel_regularizer=l1_l2(l1=0.0001, l2=0.001))(f1)  # Dense layer with L1/L2 regularization for bounding box coordinates
    regress_head = BatchNormalization()(regress_head)  # Normalize the activations in the localization head
    regress_head = Dropout(0.2)(regress_head)  # Dropout layer to reduce overfitting in the localization head
    regress_head = Dense(num_boxes * 4, activation='sigmoid')(regress_head)  # Dense layer for bounding box coordinates (x, y, w, h), scaled between 0 and 1
    regress_head = Reshape((num_boxes, 4), name='localization_output')(regress_head)  # Reshape to (num_boxes, 4) for bounding box output
    
    # Define the Model with Dual Outputs
    tracker = Model(inputs=input_layer, outputs=[class_head, regress_head])  # Create the model with classification and localization heads as outputs
    return tracker  # Return the complete model


## Test Neural Network

In [ ]:
# Instantiate the model
tracker = build_model(num_boxes=7, num_classes=3)

In [ ]:
#Shows WIZARD model architeture
tracker.summary()

In [ ]:
#Images - X, Labels - y
#Gets next batch
X, y = train.as_numpy_iterator().next()

In [ ]:
#Shows shape of Images
X.shape

In [ ]:
y[0]
# Batch class array

In [ ]:
y[1]
# Batch bounding box array

In [ ]:
# Prediction class label and bbox coordinates
classes, coords = tracker.predict(X)

In [ ]:
classes, coords
#classes - probability of crop, probability of weed, probability of background
#coords - predicted bbox coordinates

# Determine Losses & Optimizers

In [ ]:
# ADAM (Adaptive Moment Estimation) optimizer algorithm
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [ ]:
def localization_loss(y_true, yhat, delta=1.0):
    """
    Computes the Huber loss bewteen predicted and true bboxes
    while ignoring padded entries that don't represent actual bboxes.

    Args:
        y_true: Tensor of true values (ground truth), shape (batch_size, num_boxes, num_classes or 4).
        yhat: Tensor of predicted values, shape (batch_size, num_boxes, num_classes or 4).
        delta: Threshold for switching between L2 and L1 loss.

    Returns:
        Scalar tensor representing the average Huber loss over the batch.
    """
    # Create a mask for valid boxes (non-padded)
    mask = tf.reduce_any(tf.abs(y_true) > 1e-6, axis=-1)  # Adjust based on your padding representation
    mask = tf.cast(mask, dtype=tf.float32)  # Shape: (batch_size, num_boxes)

    # Calculate the absolute differences
    diff = y_true - yhat  # Shape: (batch_size, num_boxes, num_classes or 4)
    abs_diff = tf.abs(diff)  # Shape: (batch_size, num_boxes, num_classes or 4)

    # Calculate Huber loss based on the absolute differences
    quadratic_loss = 0.5 * tf.square(diff)  # Shape: (batch_size, num_boxes, num_classes or 4)
    linear_loss = delta * (abs_diff - 0.5 * delta)  # Shape: (batch_size, num_boxes, num_classes or 4)

    # Use tf.where to select the appropriate loss based on the delta condition
    loss_per_box = tf.where(abs_diff <= delta, quadratic_loss, linear_loss)  # Shape: (batch_size, num_boxes, num_classes or 4)

    # Sum the losses for each bounding box (sum across the last axis)
    loss_per_box_sum = tf.reduce_sum(loss_per_box, axis=-1)  # Shape: (batch_size, num_boxes)

    # Apply the mask to ignore padded boxes
    masked_loss = loss_per_box_sum * mask # Shape: (batch_size, num_boxes)

    # Compute the total loss by summing over all boxes and normalizing by the number of valid boxes
    total_loss = tf.reduce_sum(masked_loss) / (tf.reduce_sum(mask) + 1e-8)  # Scalar

    return total_loss


In [ ]:
# CategoricalCrossentropy: used for multi-class classification tasks
# from_logits=False: outputs are probabilities (from softmax activation)

classloss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
regressloss = localization_loss

# Train Neural Network

In [ ]:
def prepare_labels(y_classes, num_classes=3):
    """
    Converts class labels to one-hot encoding and sets padded boxes to [0, 0, 1] (Background).

    Args:
        y_classes: Tensor of shape (batch_size, num_boxes), values {0, 1, 2, 255}
        num_classes: Number of classes (including Background)

    Returns:
        Tensor of shape (batch_size, num_boxes, num_classes)
    """
    # Convert to float for manipulation
    y_classes = tf.cast(y_classes, tf.float32)
    
    # Set padded boxes to Background class (2)
    y_classes = tf.where(tf.equal(y_classes, 255), 
                         tf.ones_like(y_classes) * 2,  # Background class index
                         y_classes)
    
    # Check for invalid class labels using tf.assert
    valid_classes = tf.logical_and(tf.greater_equal(y_classes, 0), tf.less(y_classes, num_classes))
    # Use tf.debugging.assert for error checking in TensorFlow's graph mode
    tf.debugging.assert_equal(tf.reduce_all(valid_classes), True, 
                              message="Input contains invalid class labels. Must be in range [0, {}]".format(num_classes - 1))
        
    # Cast to int for one-hot encoding
    y_classes = tf.cast(y_classes, tf.int32)
    
    # One-hot encode the class labels
    y_classes = tf.one_hot(y_classes, depth=num_classes)  # Shape: (batch_size, num_boxes, num_classes)
    
    return y_classes


In [ ]:
class Tracker(Model):
    def __init__(self, tracker, **kwargs): # Initializing model "tracker"
        super().__init__(**kwargs)
        self.model = tracker # Store the model
        self.categorical_accuracy = CategoricalAccuracy(name='accuracy') # Metric for accuracy

    # Compiling model with instance variables used during training and evaluation
    def compile(self, opt, classloss, localization_loss, **kwargs): 
        super().compile(**kwargs)
        self.closs = classloss # Store the classification loss
        self.lloss = localization_loss # Store the localization loss
        self.opt = opt # Store the ADAM optimizer

    def train_step(self, batch, **kwargs): 
        X, y = batch  # X = image, y[0] = classes, y[1] = bboxes
        
        # Track gradients with tf.GradientTape
        with tf.GradientTape() as tape: 
            classes_pred, coords_pred = self.model(X, training=True) # Forward pass for predictions
            
            # Prepare labels: [1, 0, 0] for Crop, [0, 1, 0] for Weed, [0, 0, 1] for Background
            labels = prepare_labels(y[0], num_classes=3)  # Shape: (batch_size, num_boxes, 3)

            # Calculates class loss, regress loss, and total loss
            class_loss_value = self.closs(labels, classes_pred)
            localization_loss_value = self.lloss(y[1], coords_pred)
            total_loss = localization_loss_value + class_loss_value

        # Compute gradients and apply them using optimizer
        grad = tape.gradient(total_loss, self.model.trainable_variables)
        self.opt.apply_gradients(zip(grad, self.model.trainable_variables))
        # Update categorical accuracy metric
        self.categorical_accuracy.update_state(labels, classes_pred)
        
        # Return loss values and metrics
        return {
            "total_loss": total_loss, 
            "class_loss": class_loss_value, 
            "regress_loss": localization_loss_value, 
            "accuracy": self.categorical_accuracy.result()
        }

    def test_step(self, batch, **kwargs): # Predicts class and bboxes
        X, y = batch
        
        # Get predictions for both class and coordinates (bounding boxes)
        classes_pred, coords_pred = self.model(X, training=False)

        # Prepare labels: [1, 0, 0] for Crop, [0, 1, 0] for Weed, [0, 0, 1] for Background
        labels = prepare_labels(y[0], num_classes=3)  # Shape: (batch_size, num_boxes, 3)

        # Calculates class loss, regress loss, and total loss
        class_loss_value = self.closs(labels, classes_pred)
        localization_loss_value = self.lloss(y[1], coords_pred)
        total_loss = localization_loss_value + class_loss_value
        
        # Update categorical accuracy metric
        self.categorical_accuracy.update_state(labels, classes_pred)

        # Return loss values and metrics
        return {
            "total_loss": total_loss, 
            "class_loss": class_loss_value, 
            "regress_loss": localization_loss_value, 
            "accuracy": self.categorical_accuracy.result()
        }

    def call(self, X, **kwargs): 
        """
        Call the underlying model to get predictions for input X.
        Args:
            X: Input data (images).
        Returns:
            Model predictions (class and coordinates).
        """
        return self.model(X, **kwargs)


In [ ]:
# Creating instance of Tracker class model
model = Tracker(tracker)

In [ ]:
# Compiles the model with the optimizer and loss functions
model.compile(opt, classloss, regressloss)

In [ ]:
# Tensorboard toop used for visualizing metrics like loss and accuracy
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
loss_lr_scheduler = ReduceLROnPlateau(
    monitor='val_total_loss',        # Monitor validation loss
    factor=0.3,
    patience=5,
    min_lr=1e-5,
    verbose=1
)

In [ ]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_total_loss',   # Metric to monitor
    patience=8,                 # Number of epochs with no improvement after which training will stop
    restore_best_weights=True   # Restores model weights from the epoch with the best value of the monitored metric
)

In [ ]:
# Training the model with training data and validation data for a specified number of epochs
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback, early_stopping])

## Plot for Losses & Accuracy

In [ ]:
# Create a figure with 4 subplots (columns) for visualizing each metric
fig, ax = plt.subplots(ncols=4, figsize=(30, 5))

# Plot for Total Loss
# Plot training total loss with 'teal' color and label it as 'loss'
ax[0].plot(hist.history['total_loss'], color='teal', label='loss')
# Plot validation total loss with 'orange' color and label it as 'val loss'
ax[0].plot(hist.history['val_total_loss'], color='orange', label='val loss')
# Set the title for the first subplot to 'Total Loss'
ax[0].title.set_text('Total Loss')
# Display legend for the first plot
ax[0].legend()

# Plot for Classification Loss
# Plot training classification loss with 'teal' color and label it as 'class loss'
ax[1].plot(hist.history['class_loss'], color='teal', label='class loss')
# Plot validation classification loss with 'orange' color and label it as 'val class loss'
ax[1].plot(hist.history['val_class_loss'], color='orange', label='val class loss')
# Set the title for the second subplot to 'Classification Loss'
ax[1].title.set_text('Classification Loss')
# Display legend for the second plot
ax[1].legend()

# Plot for Regression Loss
# Plot training regression loss with 'teal' color and label it as 'regress loss'
ax[2].plot(hist.history['regress_loss'], color='teal', label='regress loss')
# Plot validation regression loss with 'orange' color and label it as 'val regress loss'
ax[2].plot(hist.history['val_regress_loss'], color='orange', label='val regress loss')
# Set the title for the third subplot to 'Regression Loss'
ax[2].title.set_text('Regression Loss')
# Display legend for the third plot
ax[2].legend()

# Plot for Accuracy
# Plot training accuracy with 'teal' color and label it as 'accuracy'
ax[3].plot(hist.history['accuracy'], color='teal', label='accuracy')
# Plot validation accuracy with 'orange' color and label it as 'val accuracy'
ax[3].plot(hist.history['val_accuracy'], color='orange', label='val accuracy')
# Set the title for the fourth subplot to 'Accuracy'
ax[3].title.set_text('Accuracy')
# Display legend for the fourth plot
ax[3].legend()

# Show all the plots in a single figure
plt.show()


In [ ]:
# Define a function to smooth out data points using a moving average
def smooth_curve(points, window_size=3):
    smoothed_points = []  # Initialize a list to store smoothed points
    for i in range(len(points)):
        # Determine the start of the moving window, ensuring we don't go below index 0
        start = max(0, i - window_size + 1)
        # Extract the window of points to average
        window = points[start:i + 1]
        # Calculate the average of the current window and append to smoothed points
        smoothed_points.append(sum(window) / len(window))
    return smoothed_points  # Return the list of smoothed points

# Create a figure with 4 subplots for visualizing each metric
fig, ax = plt.subplots(ncols=4, figsize=(30, 5))

# Plot for Total Loss
# Smooth and plot training total loss with 'teal' color
ax[0].plot(smooth_curve(hist.history['total_loss']), color='teal', label='loss')
# Smooth and plot validation total loss with 'orange' color
ax[0].plot(smooth_curve(hist.history['val_total_loss']), color='orange', label='val loss')
# Set the title for the first subplot to 'Total Loss'
ax[0].title.set_text('Total Loss')
# Display legend for the first plot
ax[0].legend()

# Plot for Classification Loss
# Smooth and plot training classification loss with 'teal' color
ax[1].plot(smooth_curve(hist.history['class_loss']), color='teal', label='class loss')
# Smooth and plot validation classification loss with 'orange' color
ax[1].plot(smooth_curve(hist.history['val_class_loss']), color='orange', label='val class loss')
# Set the title for the second subplot to 'Classification Loss'
ax[1].title.set_text('Classification Loss')
# Display legend for the second plot
ax[1].legend()

# Plot for Regression Loss
# Smooth and plot training regression loss with 'teal' color
ax[2].plot(smooth_curve(hist.history['regress_loss']), color='teal', label='regress loss')
# Smooth and plot validation regression loss with 'orange' color
ax[2].plot(smooth_curve(hist.history['val_regress_loss']), color='orange', label='val regress loss')
# Set the title for the third subplot to 'Regression Loss'
ax[2].title.set_text('Regression Loss')
# Display legend for the third plot
ax[2].legend()

# Plot for Accuracy
# Smooth and plot training accuracy with 'teal' color
ax[3].plot(smooth_curve(hist.history['accuracy']), color='teal', label='accuracy')
# Smooth and plot validation accuracy with 'orange' color
ax[3].plot(smooth_curve(hist.history['val_accuracy']), color='orange', label='val accuracy')
# Set the title for the fourth subplot to 'Accuracy'
ax[3].title.set_text('Accuracy')
# Display legend for the fourth plot
ax[3].legend()

# Show all the plots in a single figure
plt.show()


# Test Set Predictions

In [ ]:
def plot_predictions_with_background(yhat, test_sample, iou_threshold=0.5, score_threshold=0.5):
    
    """
    Plots the predictions with Non-Maximum Suppression applied and handles Background class.
    yhat: Tuple of (class_predictions, bbox_predictions)
    test_sample: Tuple of (images, labels)
    iou_threshold: IoU threshold for NMS
    score_threshold: Confidence threshold to filter boxes
    """
    
    class_preds, bbox_preds = yhat  # Unpacks the predictions into class probabilities and bounding box coordinates
    images = test_sample[0]  # Extracts images from the test sample tuple
    num_images = 4  # Set the number of images to display
    fig, ax = plt.subplots(ncols=num_images, figsize=(20, 20))  # Creates subplots to display images
    
    for img_idx in range(num_images):  # Loops through the number of images
        # Copy the current image to add bounding boxes
        sample_image = images[img_idx].copy()  # Copies the image for modification
        class_probs = class_preds[img_idx]  # Extracts the class probabilities for the current image
        bbox_coords = bbox_preds[img_idx]   # Extracts the bounding box coordinates for the current image
        label_classes = test_sample[1][0][img_idx]  # Extracts the labels for the current image

        boxes = []  # List to hold the bounding box coordinates
        scores = []  # List to hold the confidence scores for each box
        classes = []  # List to hold the class labels for each box
        
        for box_idx in range(7):  # Loops through each of the 7 boxes in the prediction
            if label_classes[box_idx] == 255:  # If the class label is 255, it's a padded box (no object)
                # Assign zero probability and coordinates to padded boxes
                class_probs[box_idx] = [0, 0, 1]  # Set class probabilities to background
                bbox_coords[box_idx] = [0, 0, 0, 0]  # Set bounding box coordinates to (0, 0, 0, 0)
                continue  # Skip plotting padded boxes

            # Extract probabilities for each class (Crop, Weed, Background)
            prob_crop, prob_weed, prob_background = class_probs[box_idx]
            
            # Determine the class with the highest probability
            if prob_background > score_threshold:  
                continue  # Skip for background case

            # Handle Crop class (only display confidence score, no bounding box)
            if prob_crop > prob_weed and prob_crop > score_threshold:  
                class_name = "Crop"  # Set the class name to Crop
                confidence = prob_crop  # Set the confidence to the crop probability
                color = (0, 255, 0)  # Set label color to green
                # Display confidence score at bottom-left corner of the image
                bottom_left_corner = (10, sample_image.shape[0] - 10)  # Coordinates for the bottom-left corner
                cv2.putText(sample_image, f'{class_name}: {confidence:.2f}', bottom_left_corner, 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)  # Add text to the image
                continue  # Skip drawing a bounding box for Crop

            # Handle Weed class (display bounding box with score)
            elif prob_weed > prob_crop and prob_weed > score_threshold: 
                class_name = "Weed"  # Set the class name to Weed
                confidence = prob_weed  # Set the confidence to the weed probability
                color = (255, 0, 0)  # Set the bounding box color to red
                
                # Append bounding box coordinates, confidence score, and class to lists for selected boxes
                boxes.append(bbox_coords[box_idx])
                scores.append(confidence)
                classes.append(class_name)
        
        if boxes:  # If there are any boxes to process
            boxes = np.array(boxes)  # Convert bounding boxes to NumPy array
            scores = np.array(scores)  # Convert scores to NumPy array
            classes = np.array(classes)  # Convert classes to NumPy array
            
            # Apply Non-Maximum Suppression (NMS) to reduce overlapping boxes
            selected_indices = tf.image.non_max_suppression(boxes, scores, max_output_size=5, iou_threshold=iou_threshold) 
            selected_boxes = tf.gather(boxes, selected_indices).numpy()  # Get selected bounding boxes
            selected_scores = tf.gather(scores, selected_indices).numpy()  # Get selected scores
            selected_classes = classes[selected_indices.numpy()]  # Get selected classes

            for i in range(len(selected_boxes)):  # Loop through the selected boxes after NMS
                box = selected_boxes[i]  # Get the coordinates of the selected bounding box
                class_name = selected_classes[i]  # Get the class name for the selected box
                score = selected_scores[i]  # Get the confidence score for the selected box
                color = (0, 255, 0) if class_name == "Crop" else (255, 0, 0)  # Set the color to green for Crop or red for Weed
                
                # Scale bounding boxes to the image size (224x224)
                start_point = tuple(np.multiply(box[:2], [224, 224]).astype(int))  # Calculate the top-left corner
                end_point = tuple(np.multiply(box[2:], [224, 224]).astype(int))  # Calculate the bottom-right corner
                
                # Draw the bounding box on the image (only for Weed)
                cv2.rectangle(sample_image, start_point, end_point, color, 2)  # Draw the rectangle on the image
                # Add a label with the class name and confidence score
                cv2.putText(sample_image, f'{class_name}: {score:.2f}', start_point, 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)  # Display the label on the image
        
        ax[img_idx].imshow(sample_image)  # Display the image in the corresponding subplot
        ax[img_idx].axis('on')  # Turn on the axis for the subplot
        
    plt.show()  # Display the plot

In [ ]:
# Create an iterator for the test dataset to manually retrieve batches of data
test_data = test.as_numpy_iterator()

In [ ]:
# Retrieve the next batch of data from the test iterator
test_sample = test_data.next()

In [ ]:
# Unpack the batch into features (X_test) and labels (y_test)
# Use the model to make predictions on the test batch (X_test)
X_test, y_test = test_sample
yhat = tracker.predict(X_test)

In [ ]:
#Test predictions on current model
plot_predictions_with_background(yhat, test_sample, iou_threshold=0.5, score_threshold=0.5)

In [ ]:
# Predicitions with older model
model_path = 'WIZARDv15.h5'  
tracker = load_model(model_path)

# Get the test data iterator once
test_data = test.as_numpy_iterator()

# Function to manually test a batch
def test_batch(batch_idx=0, iou_threshold=0.5, score_threshold=0.5):
    """
    Manually test a batch from the test dataset and plot the predictions.
    batch_idx: Index of the batch to test (0, 1, 2, etc.)
    """
    # Fetch the batch
    for _ in range(batch_idx):
        test_data.next()  # Skip to the desired batch
    
    # Get the next batch (or the batch at batch_idx)
    test_sample = test_data.next()
    X_test, y_test = test_sample
    
    # Run model prediction
    yhat = tracker.predict(X_test)
    
    # Plot predictions for this batch
    plot_predictions_with_background(yhat, test_sample, iou_threshold, score_threshold)

# To test a specific batch, just call:
test_batch(batch_idx=0)


In [ ]:
#Loads in any saved model
from tensorflow.keras.models import load_model

In [ ]:
#Saves the model version
tracker.save('WIZARDv15.h5')

In [ ]:
#Loads in the model for use
tracker = load_model('WIZARDv15.h5')

## Real Time Detection Code

In [ ]:
# Load the trained model
model_path = 'WIZARDv15.h5'  
tracker = load_model(model_path)  # Load the trained model using Keras' load_model function

# Define colors for bounding boxes
CROP_COLOR = (0, 255, 0)  # Green color for crop (only for text)
WEED_COLOR = (0, 0, 255)  # Red color for weeds

def preprocess_frame(frame, input_size=(224, 224)):
    """Preprocess the frame for model input."""
    frame_resized = cv2.resize(frame, input_size)  # Resize the frame to the input size expected by the model (224x224)
    frame_normalized = frame_resized / 255.0  # Normalize pixel values to the range [0, 1]
    return np.expand_dims(frame_normalized, axis=0)  # Add batch dimension (expand to shape (1, 224, 224, 3))

def draw_bounding_boxes(frame, yhat, iou_threshold=0.5, score_threshold=0.75):
    """Draw bounding boxes on the frame using model predictions."""
    class_preds, bbox_preds = yhat  # Extract class predictions and bounding box predictions from the model's output
    sample_image = frame.copy()  # Create a copy of the frame to draw on

    class_probs = class_preds[0]  # Shape: (7, 3) - Class probabilities for 7 boxes
    bbox_coords = bbox_preds[0]   # Shape: (7, 4) - Bounding box coordinates for 7 boxes

    boxes = []  # List to store valid bounding boxes
    scores = []  # List to store confidence scores for bounding boxes
    classes = []  # List to store class labels for bounding boxes

    # Iterate through each bounding box prediction
    for box_idx in range(7):  # Assuming there are 7 bounding boxes predicted per image
        # Extract the probabilities for each class (Crop, Weed, Background)
        prob_crop, prob_weed, prob_background = class_probs[box_idx]

        # For Crop: do not display bounding box, only show confidence score at bottom left
        if prob_crop > prob_weed and prob_crop > score_threshold:
            class_name = "Crop"  # Label the object as "Crop"
            confidence = prob_crop  # Set confidence score to crop's probability
            color = CROP_COLOR  # Set color to green for crop
            continue  # Skip adding bounding box for Crop, handled separately

        # For Weed: display bounding box with confidence score
        elif prob_weed > prob_crop and prob_weed > score_threshold:
            class_name = "Weed"  # Label the object as "Weed"
            confidence = prob_weed  # Set confidence score to weed's probability
            color = WEED_COLOR  # Set color to red for weed
            boxes.append(bbox_coords[box_idx])  # Add bounding box coordinates to the list
            scores.append(confidence)  # Add confidence score to the list
            classes.append(class_name)  # Add class label to the list

        else:
            continue  # Skip low-confidence or background detections

    if boxes:  # If there are valid boxes to display
        boxes = np.array(boxes)  # Convert list of boxes to a numpy array
        scores = np.array(scores)  # Convert list of scores to a numpy array
        classes = np.array(classes)  # Convert list of classes to a numpy array

        # Apply Non-Maximum Suppression to remove overlapping boxes
        selected_indices = tf.image.non_max_suppression(boxes, scores, max_output_size=5, iou_threshold=iou_threshold)
        selected_boxes = tf.gather(boxes, selected_indices).numpy()  # Get boxes that were selected after NMS
        selected_scores = tf.gather(scores, selected_indices).numpy()  # Get scores for selected boxes
        selected_classes = classes[selected_indices.numpy()]  # Get class labels for selected boxes

        # Draw each box that was kept after NMS
        for i in range(len(selected_boxes)):
            box = selected_boxes[i]  # Get coordinates of the bounding box
            class_name = selected_classes[i]  # Get the class name of the object
            score = selected_scores[i]  # Get confidence score
            color = WEED_COLOR  # Set color to red for weed

            # Scale bounding boxes back to original frame size
            frame_height, frame_width = frame.shape[:2]  # Get the dimensions of the original frame
            start_point = tuple(np.multiply(box[:2], [frame_width, frame_height]).astype(int))  # Top-left corner
            end_point = tuple(np.multiply(box[2:], [frame_width, frame_height]).astype(int))  # Bottom-right corner

            # Draw the bounding box on the frame for Weed
            if class_name == "Weed":  # Only draw box for "Weed" class
                cv2.rectangle(frame, start_point, end_point, color, 2)  # Draw rectangle
                cv2.putText(frame, f'{class_name}: {score:.2f}', start_point,  # Add text showing class and score
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2, cv2.LINE_AA)

    # Display Crop confidence score at the bottom left of the frame (no bounding box)
    for box_idx in range(7):
        prob_crop = class_probs[box_idx][0]  # Get crop class probability
        if prob_crop > score_threshold:  # If crop probability exceeds the threshold
            cv2.putText(frame, f'Crop: {prob_crop:.2f}', (10, frame.shape[0] - 10),  # Display crop score at bottom left
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, CROP_COLOR, 2, cv2.LINE_AA)
            break  # Only display one Crop confidence score (for the first detected crop)

    return frame  # Return the frame with bounding boxes and labels

def real_time_detection(model):
    """Function to run real-time object detection with webcam using the preloaded `tracker` model."""
    cap = cv2.VideoCapture(0)  # Open the webcam (0 is the default camera)

    # Check if the webcam is opened successfully
    if not cap.isOpened():
        print("Error: Could not open webcam.")  # If there's an error, print and return
        return

    # Set up the video writer to save the output
    now = datetime.now().strftime('%Y%m%d_%H%M%S')  # Get current timestamp for file naming
    output_filename = f'C:\\Users\\wfenn\\ECEN_403\\RealTimeObjectDetection\\Real_Time_Tests\\{model_path.split(".")[0]}_{now}.mp4'  # File path for output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Video codec (MP4)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Get the frame width of the video
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Get the frame height of the video
    out = cv2.VideoWriter(output_filename, fourcc, 5.0, (frame_width, frame_height))  # Initialize video writer

    # Loop to continuously capture frames from the webcam
    while True:
        ret, frame = cap.read()  # Capture a frame from the webcam

        if not ret:  # If frame capture fails
            print("Error: Failed to capture image.")  # Print error message and break the loop
            break

        # Un-mirror the frame by flipping it horizontally
        #frame = cv2.flip(frame, 1)
        
        # Preprocess the frame for the model
        preprocessed_frame = preprocess_frame(frame)  # Preprocess the captured frame
        # Get predictions from the model
        yhat = tracker.predict(preprocessed_frame)  # Get model's output
        # Draw the bounding boxes based on the predictions
        output_frame = draw_bounding_boxes(frame, yhat)  # Get the frame with bounding boxes drawn

        # Write the frame to the output file
        out.write(output_frame)  # Save the frame with bounding boxes to the output video file

        # Display the frame with bounding boxes
        cv2.imshow('Real-Time Detection', output_frame)  # Show the frame in a window

        if cv2.waitKey(50) & 0xFF == ord('q'):  # Wait for 'q' key to exit
            break

    cap.release()  # Release the webcam
    out.release()  # Release the video writer
    cv2.destroyAllWindows()  # Close all OpenCV windows

# Start real-time detection
real_time_detection(tracker) 